In [7]:
%reload_ext autoreload
%autoreload

In [38]:
from evernote.api.client import EvernoteClient
import evernote.edam.notestore.NoteStore as NoteStore
import evernote.edam.userstore.constants as UserStoreConstants
import os
import json
import shutil
from pathlib import Path
import xml.etree.ElementTree as ET
import unittest
import argparse
from integrators import *

ModuleNotFoundError: No module named 'integrators'

In [12]:
mime_to_filetype = {'image/png': 'png',
                    'image/gif': 'gif',
                    'image/jpeg': 'jpg',
                    'application/pdf': 'pdf',
                    'audio/mpeg': 'mp3',
                    'audio/x-m4a': 'm4a'}

def clean_html(original_html):
    try:
        root = ET.fromstring(original_html)
        root.tag = 'div'

        def iterator(parent, nested=False):
            for child in reversed(parent):
                if nested:
                    if len(child) >= 1:
                        iterator(child, nested=True)
                if child.tag == 'en-media' or child.tag == 'en-crypt':
                    parent.remove(child)
                elif child.tag == 'en-todo':
                    if child.tail is None:
                        child.tail = ''
                    parent.text = 'TODO ' + child.tail
                    parent.remove(child)

        iterator(root, nested=True)

        return ET.tostring(root).decode('utf-8')
    except:
        return ""

def create_token(use_oauth_flow, sandbox):    
    if not use_oauth_flow:
        # Use a developer token for authentication
        try:
            credential_file = open('evernote/dev_token', 'r')
            token = credential_file.read()
        except:
            print('ERROR: file "dev_token" seems to be missing,')
            exit(1)
    elif os.path.isfile('evernote/oauth_token'):
        # Use saved OAuth token
        credential_file = open('evernote/oauth_token', 'r')
        token = credential_file.read()
    else:
        # Request OAuth token
        try:
            consumer_key = 'memri'
            consumer_secret_file = open('evernote/consumer_secret', 'r')
            consumer_secret = consumer_secret_file.read()
        except:
            print('ERROR: file "consumer_secret" seems to be missing')
            exit(1)
        request_token_url = 'https://evernote.com/oauth'
        client_1 = EvernoteClient(consumer_key=consumer_key, consumer_secret=consumer_secret, sandbox=sandbox)
        request_token = client_1.get_request_token('https://www.memri.io/')

        # TODO manual copy of oauth_verifier to terminal should be replaced with a callback url that
        #  directs to a simple server on the pod
        print(client_1.get_authorize_url(request_token))
        print("1) Follow the link above (Ctrl-click in most terminals)\n"
              "2) Login if you're note already, and (re)authorize the app\n"
              "3) Copy the oauth_verifier, i.e. the part from the redirect url between '&oauth_verifier=' and "
              "'&sandbox_lnb=false'")
        oauth_verifier = input("Please enter oauth_verifier: ")

        client_2 = EvernoteClient(consumer_key=consumer_key, consumer_secret=consumer_secret, sandbox=sandbox)
        access_token = client_2.get_access_token(request_token['oauth_token'],
                                                 request_token['oauth_token_secret'],
                                                 oauth_verifier,
                                                 return_full_dict=False)
        token = access_token
        credential_file = open('evernote/oauth_token', 'w')
        credential_file.write(token)

    return token

In [34]:
def download_notes_and_resources(pod_client, note_store, notebook, notes, download_resources):
    for raw_note in notes:
        # print("- NOTE %s :: %s" % (raw_note.title, raw_note.guid))
        print("- NOTE :: %s" % raw_note.guid)
        note_content = note_store.getNote(raw_note.guid, True, False, False, False)

        if note_content.resources is None:
            note_content.resources = []
        if note_content.tagGuids is None:
            note_content.tagGuids = []

        if download_resources:
            for resource in note_content.resources:
                resource_content = note_store.getResource(resource.guid, True, False, False, False)
                try:
                    file_suffix = mime_to_filetype[resource_content.mime]
                except:
                    print("File type not found: ".format(resource_content.mime))
                    file_suffix = "tbd"
                print("-- RESOURCE: %s (%s)" % (resource_content.data.bodyHash.hex(), resource_content.mime))
                
                pod_client.upload_file(bytearray(resource_content.data.body))

        note_is_deleted = note_content.deleted is not None
        json_object = {'external_id': note_content.guid,
                       '_notebooks': [notebook.name],
                       '_creator': "user",
                       '_id': note_content.guid,
                       'title': note_content.title,
                       'content': clean_html(note_content.content),
                       'deleted': note_content.deleted is not None,
                       'dateCreated': int(note_content.created / 1000),
                       'dateModified': int(note_content.updated / 1000),
                       # '_tags': list(map(lambda x: tag_names[x], note_content.tagGuids))
                       '_tags': note_content.tagGuids
        }

        pod_client.create_item(json_object, item_type='Note')

def main(pod_client):
    sandbox = pod_client.get_run_variable('use_sandbox')
    download_resources = pod_client.get_run_variable('download_resources')

    client = EvernoteClient(token=pod_client.get_run_variable('token'), 
                            sandbox=sandbox, 
                            china=False)
    user_store = client.get_user_store()
    version_ok = user_store.checkVersion("Evernote EDAMTest (Python)",
                                         UserStoreConstants.EDAM_VERSION_MAJOR,
                                         UserStoreConstants.EDAM_VERSION_MINOR)
    print("Is my Evernote API version up to date? ", str(version_ok))
    print("For user : " + str(user_store.getUser()))
    note_store = client.get_note_store()

    tags = note_store.listTags()
    print("Found ", len(tags), " tags:")
    
    tag_names = dict()
    for tag in tags:
        tag_names[tag.guid] = tag.name
        json_object = {
            'external_id': tag.guid,
            'name': tag.name
        }

        pod_client.create_item(json_object, item_type='Tag')
    print("tag dictionary", tag_names)

    note_contents = []
    notebooks = note_store.listNotebooks()
    print("Found ", len(notebooks), " notebooks:")
    for (i, notebook) in enumerate(notebooks):
        search = NoteStore.NoteFilter()
        search.ascending = False
        search.notebookGuid = notebooks[i].guid
        spec = NoteStore.NotesMetadataResultSpec()
        spec.includeTitle = True
        spec.includeNotebookGuid = True
        spec.includeTagGuids = True

        print("NOTEBOOK: {}".format(notebook.guid))

        num_notes_per_fetch = 25
        num_notes_fetched = 0

        while True:
            print('download {} notes starting at {}'.format(num_notes_per_fetch, num_notes_fetched))
            result = note_store.findNotesMetadata(search, num_notes_fetched, num_notes_per_fetch, spec)
            download_notes_and_resources(pod_client, note_store, notebook, result.notes, download_resources)

            num_notes_fetched += num_notes_per_fetch
            if num_notes_fetched >= result.totalNotes:
                break
               
 # WARNING: create_token() needs a dev_token/consumer_secret which should stay secret
 # These are the settings for this importer, would normally be set by the GUI/app/client
run_item = {
    "uid": 10,
    "download_resources": True,
    "use_sandbox": True,
    "token": create_token(True, True)
}
pod_client = DryRunPodClient(run_item=run_item)
main(pod_client)

Running with POD_FULL_ADDRESS (from default) as https://0.0.0.0:3030
RUN_VARIABLES are injected/side-loaded
RUN_VARIABLES are  {'uid': 10, 'download_resources': True, 'use_sandbox': True, 'token': 'S=s1:U=95d89:E=17b78bc8cac:C=174210b5d38:P=185:A=memri:V=2:H=d90fca24c6b8e0fb4c9276040ae61029'}
Is my Evernote API version up to date?  True
For user : User(id=613769, username='gipraruloc', email=None, name=None, timezone=None, privilege=1, created=None, updated=None, deleted=None, active=True, shardId='s1', attributes=None, accounting=Accounting(uploadLimit=62914560, uploadLimitEnd=1600585200000, uploadLimitNextMonth=62914560, premiumServiceStatus=0, premiumOrderNumber=None, premiumCommerceService=None, premiumServiceStart=None, premiumServiceSKU=None, lastSuccessfulCharge=None, lastFailedCharge=None, lastFailedChargeReason=None, nextPaymentDue=None, premiumLockUntil=None, updated=None, premiumSubscriptionNumber=None, lastRequestedCharge=None, currency=None, unitPrice=None, businessId=None